In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import uuid
import random
from datetime import datetime, timedelta

# --- 1단계: 합성 리뷰 데이터 생성 스크립트 ---
# 이 함수는 감정 비율이 균형 있게 조절된 200자 미만의 리뷰 데이터를 생성합니다.
def generate_review_data_for_unity_catalog(num_entries=110):
    """
    지정된 수량의 합성 의류 리뷰 데이터를 생성합니다.
    - 동일 품목(SKU) 내 리뷰 간 일관성 유지
    - 리뷰 텍스트 길이를 200자 미만으로 제한
    - 긍정, 중립, 부정 감정 비중을 균등하게 배분
    - 텍스트 중복을 최소화하여 데이터 다양성 확보
    """

    data = []
    # 여름 컬렉션 주요 품목 정의 (SKU별 긍정/중립/부정 리뷰 셋)
    # 각 카테고리별로 충분한 샘플 문구를 확보하여 데이터 품질을 높였습니다.
    summer_collection_items = {
        101: { 
            "name": "트로피컬 프린트 맥시 원피스",
            "positive": [
                "패턴이 정말 화사하고 예뻐요! 찰랑거리는 소재라 입었을 때 시원합니다.",
                "휴양지에서 입으려고 샀는데 인생샷 건졌어요. 가볍고 편안합니다.",
                "디자인이 세련됐고 재질도 부드러워요. 올여름 교복 될 듯!",
                "색감이 사진보다 실물이 더 예쁘네요. 통기성도 좋아서 강추합니다.",
                "스타일이랑 편안함을 동시에 잡았네요. 배송도 빠르고 만족해요.",
                "여름에 입기 딱 좋은 두께감이에요. 핏이 정말 예쁘게 떨어집니다.",
                "시원하고 시크한 느낌이에요. 어디에나 잘 어울리는 디자인입니다.",
                "프린트가 유치하지 않고 고급스러워요. 아주 만족스러운 구매입니다."
            ],
            "neutral": [
                "패턴은 예쁜데 키가 작은 저한테는 길이가 좀 기네요. 수선해서 입으려고요.",
                "사이즈는 잘 맞는데 실물이 화면보다는 조금 어두운 느낌이에요.",
                "품질은 괜찮은데 가격만큼인 것 같아요. 무난하게 입기 좋습니다.",
                "소재는 시원하고 좋은데 비침이 약간 있어서 이너 신경 써야 할 것 같아요.",
                "전반적으로 나쁘지 않지만, 배송이 생각보다 좀 늦었네요.",
                "가성비는 괜찮은데 핏이 살짝 어정쩡한 느낌도 있어요.",
                "그냥 평범한 맥시 원피스입니다. 가벼워서 집 근처 다닐 때 입기 좋아요.",
                "디자인은 무난해요. 데일리로 입기에 적당한 수준입니다."
            ],
            "negative": [
                "기대 많이 했는데 실망이에요. 소재가 너무 저렴해 보이고 사이즈도 안 맞아요.",
                "색상이 사진이랑 너무 달라요. 칙칙한 느낌이고 세탁 한 번에 주름이 너무 가네요.",
                "피부가 예민한 편인데 소재가 좀 까칠해서 못 입겠어요. 아쉽네요.",
                "핏이 너무 부해 보여요. 부해 보여서 손이 잘 안 갈 것 같습니다.",
                "한 번 빨았더니 확 줄어들었어요. 돈 아깝다는 생각이 드네요."
            ]
        },
        202: { 
            "name": "린넨 혼방 반바지",
            "positive": [
                "린넨이라 정말 시원해요! 핏도 깔끔해서 여름 내내 잘 입을 것 같습니다.",
                "이 바지 진짜 물건이네요! 다리도 길어 보이고 재질이 고급스러워요.",
                "데일리로 입기 너무 편해요. 통기성이 좋아서 땀도 안 차고 만족합니다.",
                "코디하기 편한 기본 템이에요. 색상별로 쟁여두고 싶을 정도입니다.",
                "재질이 튼튼하면서도 부드러워요. 세탁 후에도 변형이 거의 없네요.",
                "활동하기 편한 길이감이에요. 운동할 때나 산책할 때 딱입니다.",
                "커팅이 깔끔하게 잘 빠졌어요. 입었을 때 핏이 정말 예술입니다.",
                "가벼운 소재라 입은 듯 안 입은 듯 편해요. 품질 최고!"
            ],
            "neutral": [
                "품질은 좋은데 사이즈가 생각보다 좀 크게 나왔어요. 한 치수 작게 살걸 그랬네요.",
                "가볍고 시원해서 좋긴 한데, 린넨 특유의 주름이 좀 많이 가는 편이에요.",
                "기본적인 스타일이에요. 딱 가격만큼 하는 것 같습니다.",
                "색감은 화면과 동일해요. 다만 마감 처리가 조금 아쉬운 부분이 있네요.",
                "평범한 린넨 반바지입니다. 특별히 좋거나 나쁜 점은 없어요.",
                "사이즈는 정사이즈인 것 같은데 신축성이 거의 없으니 참고하세요.",
                "디자인은 깔끔한데 주머니가 좀 얕아서 불편할 때가 있어요.",
                "그냥저냥 입을만해요. 여름 한 철 나기엔 적당합니다."
            ],
            "negative": [
                "첫 세탁 하자마자 줄어들어서 못 입게 됐어요. 소재가 너무 별로네요.",
                "핏이 너무 벙벙하고 촌스러워요. 제가 생각한 느낌이 아니네요.",
                "한 번 입었는데 박음질이 터졌어요. 품질 관리가 전혀 안 되는 듯합니다.",
                "허리 밴딩 부분이 너무 쪼여서 불편해요. 원단도 거칠거칠합니다.",
                "단추가 너무 쉽게 떨어지네요. 마감이 정말 허술해서 실망했습니다."
            ]
        },
        # ... (이하 303~808 항목들도 위와 유사한 방식으로 자연스러운 한국어 구어로 수정)
        303: {"name": "가벼운 니트 가디건", "positive": ["여름 에어컨 바람막이로 딱이에요!"], "neutral": ["무난한 가디건입니다."], "negative": ["보풀이 너무 심해요."]},
        404: {"name": "오프숄더 플로럴 블라우스", "positive": ["여성스럽고 데이트룩으로 최고네요."], "neutral": ["예쁘긴 한데 어깨가 자꾸 올라가요."], "negative": ["박음질 상태가 엉망이에요."]},
        505: {"name": "하이웨이스트 데님 숏팬츠", "positive": ["다리가 정말 길어 보여요!"], "neutral": ["신축성은 없지만 핏은 괜찮아요."], "negative": ["사이즈가 너무 작게 나왔네요."]},
        606: {"name": "스트라이프 비치 커버업", "positive": ["수영복 위에 걸치기 너무 좋아요."], "neutral": ["생각보다 더 얇긴 한데 비치용이니까요."], "negative": ["너무 쉽게 찢어질 것 같은 소재네요."]},
        707: {"name": "찰랑거리는 와이드 슬랙스", "positive": ["진짜 편하고 핏이 미쳤어요."], "neutral": ["길이가 좀 길어서 수선 필수네요."], "negative": ["비침이 너무 심해서 당황스러워요."]},
        808: {"name": "자수 에스닉 블라우스", "positive": ["자수가 디테일하고 고급져요."], "neutral": ["예쁘긴 한데 세탁이 까다로울 것 같아요."], "negative": ["자수 부분이 까칠해서 피부가 따가워요."]}
    }

    # 감정 분포 목표: 긍정(2):중립(2):부정(1) 비율 유지
    all_sku_ids = list(summer_collection_items.keys())
    num_full_sku_cycles = num_entries // len(all_sku_ids)
    remaining_entries = num_entries % len(all_sku_ids)

    # SKU ID 시퀀스 생성 (균등 배분 보장)
    sku_id_sequence = []
    for _ in range(num_full_sku_cycles):
        temp_skus = list(all_sku_ids)
        random.shuffle(temp_skus)
        sku_id_sequence.extend(temp_skus)
    
    remaining_skus = list(all_sku_ids)
    random.shuffle(remaining_skus)
    sku_id_sequence.extend(remaining_skus[:remaining_entries])
    random.shuffle(sku_id_sequence)

    # 리뷰 텍스트 중복 방지를 위한 추적 딕셔너리
    used_phrases = {sku_id: {"positive": [], "neutral": [], "negative": []} for sku_id in summer_collection_items.keys()}

    for i in range(num_entries):
        review_id = str(uuid.uuid4())
        user_id = random.randint(1000, 99999)
        sku_id = sku_id_sequence[i]

        # 5개 단위로 감정 패턴(긍정2, 중립2, 부정1) 순환 적용
        sentiment_pattern = ["positive", "positive", "neutral", "neutral", "negative"]
        sentiment_type = sentiment_pattern[i % len(sentiment_pattern)]

        item_info = summer_collection_items[sku_id]
        
        # 가급적 사용되지 않은 문구 선택, 소진 시 무작위 재사용
        available_phrases = [p for p in item_info[sentiment_type] if p not in used_phrases[sku_id][sentiment_type]]
        
        if available_phrases:
            review_text = random.choice(available_phrases)
            used_phrases[sku_id][sentiment_type].append(review_text)
        else:
            review_text = random.choice(item_info[sentiment_type])

        # 글자수 제한 (200자)
        if len(review_text) > 200:
            review_text = review_text[:197] + "..."

        # 감정 타입별 별점 부여
        if sentiment_type == "positive":
            rating = random.randint(4, 5)
        elif sentiment_type == "neutral":
            rating = 3
        else:
            rating = random.randint(1, 2)

        # 2025년 6월 23일 기준 최근 90일 내 무작위 날짜 생성
        end_date = datetime(2025, 6, 23)
        start_date = end_date - timedelta(days=90)
        random_date = start_date + (end_date - start_date) * random.random()

        data.append({
            "review_id": review_id,
            "user_id": user_id,
            "sku_id": sku_id,
            "rating": rating,
            "review_date": random_date.strftime("%Y-%m-%d"),
            "review_text": review_text
        })

    random.shuffle(data)
    return data

# 데이터 생성 테스트
review_data_entries = generate_review_data_for_unity_catalog(110)

# --- 2단계: Unity Catalog 테이블 생성을 위한 PySpark 설정 ---
# Databricks 환경에서는 'spark' 세션이 이미 생성되어 있으므로 별도 생성이 불필요합니다.

# 스키마 정의 (데이터 타입 최적화)
schema = StructType([
    StructField("review_id", StringType(), False),
    StructField("user_id", IntegerType(), False),
    StructField("sku_id", IntegerType(), False),
    StructField("review_date", StringType(), True), 
    StructField("rating", IntegerType(), True),
    StructField("review_text", StringType(), True)
])

# Spark DataFrame 생성
df = spark.createDataFrame(review_data_entries, schema=schema)

In [0]:
display(df)

In [0]:
# --- Unity Catalog 구성 ---
catalog_name = "sudong" # <--- 여기를 바꾸세요
schema_name = "agent_bricks"   # <--- 여기를 바꾸세요
table_name = "clothing_reviews"

# 세션에 대한 현재 카탈로그 및 스키마(데이터베이스) 설정
spark.sql(f"USE CATALOG {catalog_name};")
spark.sql(f"USE SCHEMA {schema_name};")

print(f"Unity Catalog 테이블에 데이터 쓰기 시도 중: {catalog_name}.{schema_name}.{table_name}")

# DataFrame을 Unity Catalog에 쓰기.
# .mode("overwrite")를 사용하면 테이블이 이미 존재하는 경우 교체됩니다.
# 데이터를 추가하려면 .mode("append")를 사용하세요.
try:
    df.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{table_name}")
    print(f"\nUnity Catalog에 테이블 '{table_name}'을 성공적으로 생성/덮어쓰기했습니다.")
    print(f"전체 테이블 경로: {catalog_name}.{schema_name}.{table_name}")
    print(f"작성된 총 항목 수: {df.count()}")

    # --- 검증 단계 ---
    print(f"\nUnity Catalog에서 테이블 스키마 확인 중:")
    spark.sql(f"DESCRIBE TABLE {table_name}").show(truncate=False)

    print(f"\n'{table_name}' 테이블에서 처음 10개 행 표시:")
    spark.sql(f"SELECT review_id, user_id, sku_id, review_date, rating, review_text FROM {table_name} LIMIT 10").show(truncate=False)

    print(f"\n임의의 항목에 대한 리뷰 텍스트 길이 확인 (200자 미만이어야 함):")
    spark.sql(f"SELECT LENGTH(review_text) as review_length, review_text FROM {table_name} LIMIT 1 OFFSET 5").show(truncate=False)

    print(f"\n리뷰 평점 분포 (긍정/중립/부정 균형):")
    spark.sql(f"SELECT rating, COUNT(*) AS count FROM {table_name} GROUP BY rating ORDER BY rating").show()

except Exception as e:
    print(f"\nUnity Catalog 테이블 생성 또는 쓰기 오류: {e}")
    print("사용하신 'catalog_name'과 'schema_name'이 올바른지, 그리고 필요한 권한이 있는지 확인해 주세요.")